In [4]:
from battle import *
from trainer import *
from pokemon import *
from moves import *
from globals import *
from team import *

In [5]:
moves=[Tackle(), Earthquake()]
pikachuStats={
    Stat.HP : 200,
    Stat.ATT : 100,
    Stat.DEF : 100,
    Stat.SPA : 100,
    Stat.SPD : 100,
    Stat.SPE : 100
}

ratStats={
    Stat.HP : 200,
    Stat.ATT : 100,
    Stat.DEF : 100,
    Stat.SPA : 100,
    Stat.SPD : 100,
    Stat.SPE : 101
}
pikachu=Pikachu('pika', 15, pikachuStats, moves)
rattata=Rattata('rat', 20, ratStats, moves)
t1=Trainer('Trainer 1', [pikachu], None)
t2=Trainer('Trainer 2', [rattata], None)
teams=[Team('Team Pikachu', [t1], 1), Team('Team Rattata', [t2], 1)]
testBattle=Battle(teams)

In [6]:
testBattle.runBattle()
# TODO: fix pokemon still attacking from the grave
# TODO: sync pokemon fainted status with corresponding slot being empty
# TODO: make pokemon status an enum(benched, fainted, active)
# TODO: only show trainers when swapping and sending out pokemon if they have pokemon available

List of possible trainers: 
	 0 Trainer 1
List of possible pokemon: 
	 0 pika
pika was selected!


List of possible trainers: 
	 0 Trainer 2
List of possible pokemon: 
	 0 rat
rat was selected!


Team Pikachu
----
pika
200
----


Team Rattata
---
rat
200
---


Trainer 1 is picking a move for pika
List of moves: 2
	 0 Tackle
	 1 Earthquake
Earthquake was selected.


Team Team Pikachu's Pokemon pika is choosing a target!
0 Pokemon from Team Team Pikachu:
0 pika
1 Pokemon from Team Team Rattata:
0 rat
Team Team Rattata was selected as a target!
rat was selected as a target!


Trainer 2 is picking a move for rat
List of moves: 2
	 0 Tackle
	 1 Earthquake
Earthquake was selected.


Team Team Rattata's Pokemon rat is choosing a target!
0 Pokemon from Team Team Pikachu:
0 pika
1 Pokemon from Team Team Rattata:
0 rat
Team Team Pikachu was selected as a target!
pika was selected as a target!


Earthquake was super effective!
pika took 200.0 damage!
pika has fainted!


1